In [1]:
import time

import scipy as sp
import numpy as np
import librosa
import matplotlib.pyplot as plt
from datetime import datetime

from modules.lpc import toeplitz_solver
from modules.autocorrelation import autocorrelation

In [2]:
TIME_TEMPLATE = '%Y%m%d%H%M%S'

In [3]:
def calc_process_time(args, func):
    start = time.perf_counter()
    func(**args)
    end = time.perf_counter()

    return end - start

In [9]:
wave_data, sr = librosa.load('./audio/aiueo.wav')
ac = autocorrelation(wave_data)

In [10]:
# a, e = toeplitz_solver(ac, dim)
sp_res = calc_process_time(
    {'ac': ac, 'dim': 100}, toeplitz_solver)

In [11]:
sp_res

0.003165391999999656

In [16]:
# _ac = ac[:100]
sp_res = calc_process_time(
    {'c_or_cr': (ac[:100], ac[:100]), 'b': ac[1:101]}, sp.linalg.solve_toeplitz)

In [17]:
sp_res

0.02775825500003748

In [ ]:
length = [
    22050*1,
    22050*10,
    22050*50,
    22050*100,
#     22050*150,
#     22050*500,
#     22050*1000,
#     22050*1500,
]
result = []

for l in length:
    print(l)
    fake_data = 2 * np.random.rand(l) - 1
    ac = autocorrelation(fake_data)

    res = {
        'scipy_solver': [],
        'mine_solver': []
    }

    for i in range(10):
        sp_res = calc_process_time(
            {'c_or_cr': (ac[:-1], ac[:-1]), 'b': ac[1:]},
            sp.linalg.solve_toeplitz
        )
        res['scipy_solver'].append(sp_res)
        
        mine_res = calc_process_time({'ac': ac, 'dim': len(ac)-1}, toeplitz_solver)
        res['mine_solver'].append(mine_res)


    result.append(res)

22050
220500


In [ ]:
aves = {
    'scipy_solver': [],
    'mine_solver': []
}
for i, res in enumerate(result):
    print('-'*10)
    for k, v in res.items():
        print(k, sum(v)/len(v))
        aves[k].append(sum(v)/len(v))
print('-'*10)

In [ ]:
p_scipy = plt.plot(length, aves['scipy_solver'], marker="o", label='scipy')
p_mine = plt.plot(length, aves['mine_solver'], marker="o", label='mine')
plt.legend(loc=2)
timestamp = datetime.now().strftime(TIME_TEMPLATE)
plt.savefig('bench-res-{}.png'.format(timestamp))